In [13]:
import numpy as np
import scipy.stats
import torch
confidence = 0.95
class Value:
	'''A measured or calculated value, with uncertainty.'''
	def __init__(self, value, uncertainty=0):
		self.value = float(value)
		self.uncertainty = float(uncertainty)

	def __str__(self) -> str:
		return f'{self.value} ± {self.uncertainty}'

	def __repr__(self) -> str:
		return f'Value({self.value}, {self.uncertainty})'

	def relative_uncertainty(self) -> float:
		'''calculate the relative uncertainty'''
		return self.uncertainty / self.value

In [14]:
def margin_of_error(value : np.array) -> float:
	'''calculate Margin of Error (the uncertainty associated with random error), using the t-distribution'''
	return scipy.stats.t.ppf((1 + confidence) / 2, len(value) - 1) * np.std(value, ddof=1) / np.sqrt(len(value))

def measure(value : np.array, tolerance=0, factor=1, intercept=0) -> Value:
	'''measure a value with uncertainty, assuming the data is uniformly distributed within the tolerance'''
	systematic_error = tolerance * confidence
	if hasattr(value, '__len__'):
		ME = margin_of_error(value)
		return Value((np.mean(value) - intercept) * factor, np.sqrt(ME**2 + systematic_error**2) * factor)
	else:
		return Value((value - intercept) * factor, systematic_error * factor)

def calc(f, *args : float | Value) -> Value:
	'''calculate a function with uncertainty, using torch'''
	# Convert all arguments to torch tensors
	values = [
		torch.tensor(
			arg.value if isinstance(arg, Value) else arg,
			requires_grad=True,
			dtype=float
		)
		for arg in args
	]
	uncertainties = [torch.tensor(arg.uncertainty if isinstance(arg, Value) else 0) for arg in args]

	# Calculate the function value
	value : torch.Tensor = f(*values)

	# Calculate the uncertainty using the formula for error propagation
	value.backward()
	uncertainties = torch.Tensor([(uncertainty * value.grad) for uncertainty, value in zip(uncertainties, values)])
	uncertainty = torch.sqrt(torch.sum(uncertainties ** 2))
	calc.uncertainties = torch.abs(uncertainties)

	return Value(value.detach().numpy(), uncertainty.detach().numpy())

# Experiment 2 - Measuring the viscosity of a liquid using falling ball method

In [15]:
def yita(r, row_2, row_1, g, t, s, R):
	return 2 / 9 * r ** 2 * (row_2 - row_1) * g * t / s / (1 + 2.4 * r / R)

def row(m, r):
	return m / (4 / 3 * torch.pi * r ** 3)

In [16]:
def f(s_1, s_2, t, m, g, d, D, row_1):
	r = d / 2
	R = D / 2
	row_2 = row(m, r)
	s = s_2 - s_1
	return yita(r, row_2, row_1, g, t, s, R)


In [17]:
def steel_ruler(l): return measure(l, 0.4, 1e-3)
s_1 = steel_ruler(26.5)
s_2 = steel_ruler(169.0)
# s: the distance between the laser beams

t = measure(
	[12.07, 12.10, 11.97, 11.81, 11.66, 11.87, 11.72, 11.66, 11.68, 11.63,
  	 11.66, 11.69, 11.44, 11.28, 11.43, 11.47, 11.28, 11.72, 11.25, 11.22],
  0.005
)
# t: the time it takes for the steel ball to fall through the laser beams

In [18]:
m = measure(3.277, 0.0005, 1e-3 / 100)
# m: the mass of the steel ball, measured with 100 balls

g = 9.793
# g: the acceleration due to gravity, treated as a constant

def micrometer(l): return measure(l, 0.002, 1e-3, 0.00)
d = micrometer(
	[1.992, 1.991, 1.991, 1.990, 1.990, 1.990, 1.989, 1.991, 1.990, 1.989]
)
# d: the diameter of the steel ball

In [19]:
def vernier_caliper(l): return measure(l, 0.002, 1e-2, 0.006)
D = vernier_caliper(
	[6.116, 6.140, 6.128, 6.148, 6.140, 6.132]
)
# D: the diameter of the measuring cylinder

row_1 = measure(0.590, 0.004, 1e3)
# row_1: the density of the caster oil
temperature = measure(17.5, 0.5)
# temperature: the temperature of the caster oil
humidity = measure(64.0, 0.5)
# humidity: the humidity of the lab

In [25]:
t, d, D

(Value(11.630500000000001, 0.12271451325353376),
 Value(0.0019903000000000004, 2.0175633701583273e-06),
 Value(0.06128, 0.00011950574740097389))

In [26]:
t.relative_uncertainty(), d.relative_uncertainty(), D.relative_uncertainty()

(0.010551095245564142, 0.0010136981209658477, 0.0019501590633318193)

In [20]:
row_1

Value(590.0, 3.8)

In [21]:
calc((lambda m, d : row(m, d / 2)), m, d)

Value(7938.202804991214, 24.168230056762695)

In [22]:
(result := calc(f, s_1, s_2, t, m, g, d, D, row_1))

Value(1.199076087086592, 0.013546465896070004)

In [23]:
result.relative_uncertainty()

0.011297419773405704

In [24]:
calc.uncertainties

tensor([0.0032, 0.0032, 0.0127, 0.0002, 0.0000, 0.0016, 0.0002, 0.0006])

In [27]:
calc.uncertainties / result.uncertainty

tensor([0.2360, 0.2360, 0.9339, 0.0139, 0.0000, 0.1178, 0.0125, 0.0458])